In [1]:
import pandas as pd
from zipfile import ZipFile
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
# from imblearn.over_sampling import SMOTE
from collections import Counter
import numpy as np

# Data Preparation (done from past weeks)

In [2]:
zip_file = ZipFile('../../data/raw/forStudents.pkl.zip')
df = pd.read_pickle(zip_file.open('forStudents.pkl'))
df

,acquisition_type,snapshot_type,channel,evaluation_dt,vintage,is_app_approved,vantage3_score,bad,bad_v2,bad_balance,...,rtr5930,rtr7110,rtr7140,rtr7160,rtr7216,rtr7620,rtr8120,rtr8220,rtr8320,rowNumber
0,ORGANIC,FLOW_MODULARITY,OTHER,2021-08-04,202108,1,698.0,0.0,0.0,0.0,...,2.0,4.0,4.0,9.0,5.0,100.0,5.0,5.0,5.0,59486
1,ORGANIC,FLOW_MODULARITY,OTHER,2021-03-01,202103,1,731.0,0.0,0.0,0.0,...,30.0,2.0,2.0,130.0,12.0,100.0,4.0,4.0,4.0,2357
2,ORGANIC,FLOW_MODULARITY,OTHER,2021-02-26,202102,1,NaN,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90398
3,ORGANIC,FLOW_MODULARITY,OTHER,2021-06-04,202106,1,715.0,0.0,0.0,0.0,...,249.0,18.0,18.0,56.0,7.0,100.0,5.0,151.0,45.0,104654
4,ORGANIC,FLOW_MODULARITY,OTHER,2021-07-14,202107,1,NaN,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48707,ORGANIC,FLOW_MODULARITY,OTHER,2021-05-01,202105,1,732.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,131881
48708,ORGANIC,FLOW_MODULARITY,OTHER,2020-11-29,202011,1,705.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,47197
48709,ORGANIC,FLOW_MODULARITY,OTHER,2021-04-25,202104,1,722.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,0.0,NaN,2.0,167.0,86.0,29739
48710,ORGANIC,FLOW_MODULARITY,OTHER,2021-04-10,202104,1,NaN,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89207


In [3]:
# Clean and imputation

df_clean = df[df['all0000'].notna()]
df_clean = df_clean[df_clean['bad'] != -1.0]
df_clean = df_clean[df_clean['is_app_approved'] == 1]
df_clean.drop(columns=['bad_v2','evaluation_dt','vintage', "vantage3_score", "bad_balance", "current_balance", "net_spend"], inplace=True)
described = df_clean["annual_income"].describe()
quartiles = list(described.iloc[4:7])

In [4]:
def assign_quartile(row):
    income = row["annual_income"]
    if income < quartiles[0]:
        return 0
    elif ((quartiles[0] < income) & (quartiles[1] >= income)):
        return 1
    elif ((quartiles[1] < income) & (quartiles[2] >= income)):
        return 2
    else:
        return 3

In [5]:
df_clean.apply(assign_quartile, axis=1).value_counts().sort_index()

0    10625
1    10546
2    10709
3    10667
dtype: int64

In [6]:
df_clean['income_quartile'] = df_clean.apply(assign_quartile, axis=1)

In [7]:
dfs = []
for i in range(4):
    quartile = df_clean.loc[df_clean["income_quartile"] == i]
    dfs.append(quartile.fillna(quartile.mean()))
    
df_clean = pd.concat(dfs)

temp = df_clean.isna().sum().to_frame()
temp[temp[0] != 0]

df_clean.drop(columns=['all9230', 'all9240', 'all9249', 'all9280'], inplace=True)
df_clean = df_clean.fillna(df_clean.mean())

/var/folders/bg/tjm2hhbn7vl_j003610q1dnc0000gn/T/ipykernel_81642/448997779.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  dfs.append(quartile.fillna(quartile.mean()))
/var/folders/bg/tjm2hhbn7vl_j003610q1dnc0000gn/T/ipykernel_81642/448997779.py:12: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_clean = df_clean.fillna(df_clean.mean())


In [8]:
# Categorical

def process_categorical_data(dataset):
    """ One hot encodes all of the categorial columns of the dataset.  Removes the original columns """
    # select categorical data
    categorical_data = dataset.select_dtypes('object')
    
    enc = OneHotEncoder(handle_unknown='ignore')
    
    ohe_data = enc.fit_transform(categorical_data).toarray()
    column_names = enc.get_feature_names(categorical_data.columns)
    
    add = dataset.select_dtypes(exclude='object')
    added = pd.DataFrame(ohe_data,columns=column_names).astype(int)
    
    add.reset_index(drop = True, inplace= True)
    added.reset_index(drop = True, inplace= True)
    X = pd.concat([add, 
               added], axis=1)
    print(X.shape)
    return X

df_clean = process_categorical_data(df_clean)
df_clean

/Users/kylenero/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


(42547, 887)


,is_app_approved,bad,annual_income,all0000,all0060,all0061,all0101,all0200,all0201,all0206,...,state_code_SD,state_code_TN,state_code_TX,state_code_UT,state_code_VA,state_code_VT,state_code_WA,state_code_WI,state_code_WV,state_code_WY
0,1,0.0,15171,10.0,8.000000,8.000000,0.0,9.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
1,1,0.0,20000,3.0,4.165954,5.237061,0.0,3.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,1,0.0,7000,6.0,4.165954,5.237061,0.0,4.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,1,1.0,18000,3.0,1.000000,1.000000,0.0,2.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,1,0.0,30000,6.0,4.165954,5.237061,0.0,2.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42542,1,0.0,125000,12.0,9.305257,11.139763,0.0,11.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
42543,1,0.0,101000,60.0,21.000000,11.139763,0.0,43.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
42544,1,0.0,145000,53.0,9.305257,11.139763,0.0,52.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
42545,1,0.0,72000,7.0,9.305257,11.139763,0.0,7.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0


# Week 4-6 starts here:

### Brainstorm with your team some ideas of new features you can create.

We are going to attempt to reduce the dimensionality of our features. PCA can remove correlated features, reduce overfitting, and thus improve the accuracy of our ML model.

In [9]:
df_clean

,is_app_approved,bad,annual_income,all0000,all0060,all0061,all0101,all0200,all0201,all0206,...,state_code_SD,state_code_TN,state_code_TX,state_code_UT,state_code_VA,state_code_VT,state_code_WA,state_code_WI,state_code_WV,state_code_WY
0,1,0.0,15171,10.0,8.000000,8.000000,0.0,9.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
1,1,0.0,20000,3.0,4.165954,5.237061,0.0,3.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,1,0.0,7000,6.0,4.165954,5.237061,0.0,4.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,1,1.0,18000,3.0,1.000000,1.000000,0.0,2.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,1,0.0,30000,6.0,4.165954,5.237061,0.0,2.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42542,1,0.0,125000,12.0,9.305257,11.139763,0.0,11.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
42543,1,0.0,101000,60.0,21.000000,11.139763,0.0,43.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
42544,1,0.0,145000,53.0,9.305257,11.139763,0.0,52.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
42545,1,0.0,72000,7.0,9.305257,11.139763,0.0,7.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0


In [10]:
# balancing df clean using undersampling method
df_clean_0 = df_clean[df_clean['bad'] == 0.0]
df_clean_1 = df_clean[df_clean['bad'] == 1.0]

df_clean_0_sample = df_clean_0.sample(n=len(df_clean_1), replace=False, random_state = 42)
df_clean_balanced = pd.concat([df_clean_0_sample, df_clean_1])

len(df_clean_balanced) # should be 3693 * 2 = 7386

7386

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [22]:
def model_evaluator(data):
    X = data.drop('bad', axis=1)
    y = data['bad'].astype(int)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    clf = LogisticRegression(penalty='none').fit(X_train,y_train)
    y_pred =clf.predict(X_test)
    
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    
    X_test['predicted_bad'] = y_pred
#     print("Dollar Loss: " + str(X_test[(y_test == 1) & (y_test == y_pred)]['bad_balance'].sum().round(2)))

In [23]:
model_evaluator(df_clean_balanced)

[[510 604]
 [319 783]]
              precision    recall  f1-score   support

           0       0.62      0.46      0.52      1114
           1       0.56      0.71      0.63      1102

    accuracy                           0.58      2216
   macro avg       0.59      0.58      0.58      2216
weighted avg       0.59      0.58      0.58      2216



/Users/kylenero/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [24]:
X = df_clean_balanced.drop('bad', axis=1)
y = df_clean_balanced['bad'].astype(int)

In [25]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
scaled_data = pd.DataFrame(scaler.transform(X))

In [26]:
scaled_data

,0,1,2,3,4,5,6,7,8,9,...,876,877,878,879,880,881,882,883,884,885
0,0.0,-0.063132,-0.271052,-0.450030,-0.565297,-0.094785,-0.392699,-0.02716,-0.22507,-0.203263,...,-0.04034,-0.1328,4.098486,-0.072858,-0.158946,-0.053398,-0.152097,-0.130672,-0.088965,-0.04034
1,0.0,-0.057225,-0.723091,-0.450030,-0.565297,-0.094785,-0.682158,-0.02716,-0.22507,-0.607925,...,-0.04034,-0.1328,-0.243993,-0.072858,-0.158946,-0.053398,-0.152097,-0.130672,-0.088965,-0.04034
2,0.0,-0.026283,-0.271052,-1.298225,0.233685,-0.094785,-0.199727,-0.02716,-0.22507,0.066512,...,-0.04034,-0.1328,-0.243993,-0.072858,-0.158946,-0.053398,-0.152097,-0.130672,-0.088965,-0.04034
3,0.0,-0.026521,0.271395,-0.483818,0.233685,-0.094785,0.282705,-0.02716,-0.22507,0.740950,...,-0.04034,-0.1328,-0.243993,-0.072858,-0.158946,-0.053398,-0.152097,-0.130672,-0.088965,-0.04034
4,0.0,-0.034613,-0.451868,-0.280217,-0.616574,-0.094785,-0.392699,-0.02716,-0.22507,-0.338150,...,-0.04034,-0.1328,-0.243993,-0.072858,-0.158946,-0.053398,-0.152097,-0.130672,-0.088965,-0.04034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7381,0.0,0.027272,0.361803,1.144995,0.897553,-0.094785,0.379192,-0.02716,-0.22507,0.201400,...,-0.04034,-0.1328,-0.243993,-0.072858,-0.158946,-0.053398,-0.152097,-0.130672,-0.088965,-0.04034
7382,0.0,-0.002480,0.542618,0.126987,-0.183967,-0.094785,-0.585672,-0.02716,-0.22507,-0.607925,...,-0.04034,-0.1328,-0.243993,-0.072858,-0.158946,-0.053398,-0.152097,-0.130672,-0.088965,-0.04034
7383,0.0,0.033223,1.265881,0.534190,0.248641,-0.094785,1.440542,-0.02716,-0.22507,0.606062,...,-0.04034,-0.1328,-0.243993,-0.072858,-0.158946,-0.053398,-0.152097,-0.130672,-0.088965,-0.04034
7384,0.0,-0.009621,0.180987,-0.076615,-0.400271,-0.094785,0.282705,-0.02716,-0.22507,0.336287,...,-0.04034,-0.1328,-0.243993,-0.072858,-0.158946,-0.053398,-0.152097,-0.130672,-0.088965,-0.04034


In [27]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
pca = PCA(n_components=50)
pca.fit(scaled_data)
transformed_data = pca.fit_transform(scaled_data)
pca_data = pd.DataFrame(transformed_data)

In [28]:
pca_data

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,-7.177256,-4.512930,4.265165,2.081165,-2.684958,-1.721771,-2.413139,4.802517,-0.447682,-1.828026,...,0.506034,-0.459289,0.615877,0.151307,-1.442349,1.032909,0.876568,1.323530,0.754490,-0.710971
1,-8.489437,-0.076108,4.600491,1.496669,1.943936,-1.350314,-0.997756,3.757002,-1.257632,-3.831396,...,0.175949,1.059862,0.016989,-0.279196,1.695105,0.199494,0.524844,-0.929142,0.529345,0.287508
2,-4.528945,2.689593,-1.525833,3.402564,-10.939200,32.022138,1.318511,4.352358,-2.980177,-0.087490,...,2.829476,-2.259185,4.835632,0.218144,-0.144548,-7.421301,-5.855151,-3.343002,4.192284,2.040897
3,8.412768,13.358951,-0.496734,0.207961,2.105097,0.205440,3.957844,-4.745018,-5.984198,-5.472680,...,-1.974302,2.905381,-0.234109,1.306551,-0.394813,-3.620326,0.787194,-3.218356,2.361230,-1.342282
4,-3.332479,6.545506,1.212955,-3.897411,-0.315443,0.101074,-5.376316,-5.889098,-0.887423,-4.343383,...,1.529428,-1.646257,0.207335,-0.928061,-0.525641,0.074500,-0.895312,0.136226,1.262721,0.115592
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7381,8.550572,-4.179188,0.556905,-4.050598,13.125278,0.769346,-1.940288,3.237755,-2.061999,1.792705,...,-0.191181,2.613030,-5.353978,-1.484518,-1.396611,-1.559217,-0.967333,1.818219,1.118209,0.496174
7382,-0.361224,-1.344797,-5.767633,-4.212039,-3.160671,-1.235665,-1.471936,0.741608,0.056694,1.520996,...,-1.470034,-2.974942,-1.759671,-1.520050,0.567743,-5.092682,1.738637,-0.136290,2.435627,1.340647
7383,8.440756,12.255782,-7.983140,-3.144337,-4.322580,-4.804486,0.112826,3.336191,-5.525658,-1.816126,...,-1.044873,1.781794,1.179819,0.202201,-0.332641,1.835887,2.028490,2.067713,-0.475001,-0.973368
7384,8.333123,-1.642463,-5.931626,-3.075135,6.921080,2.412134,-1.667396,-0.803672,0.836670,4.359455,...,0.226390,0.421053,-0.275075,0.868860,-1.083759,4.208262,-0.349237,0.003363,2.935929,0.740177


In [29]:
def pca_model(new_x, new_y):
    X_train, X_test, y_train, y_test = train_test_split(new_x, new_y, test_size=0.3, random_state=42)

    clf = LogisticRegression(penalty='none').fit(X_train,y_train)
#     clf = RandomForestClassifier(max_depth = 15, random_state=42).fit(X_train, y_train)
    y_pred =clf.predict(X_test)
    
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    
    X_test['predicted_bad'] = y_pred
#     print("Dollar Loss: " + str(X_test[(y_test == 1) & (y_test == y_pred)]['bad_balance'].sum().round(2)))

In [30]:
pca_model(pca_data, y)

[[676 438]
 [400 702]]
              precision    recall  f1-score   support

           0       0.63      0.61      0.62      1114
           1       0.62      0.64      0.63      1102

    accuracy                           0.62      2216
   macro avg       0.62      0.62      0.62      2216
weighted avg       0.62      0.62      0.62      2216



In [31]:
# accuracy went from 0.58 to 0.62 after pca! nice! also, f1-score for the 0 class improved.